<a href="https://colab.research.google.com/github/a-r-soares/desafio_etl_python/blob/main/DesafioETLPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
import json
import time
import random

# --- CONFIGURAÇÃO ---
GOOGLE_API_KEY = "AI_STUDIO_API_KEY_AQUIm-MMbVxZOej1BynUk8" # <--- INSIRA SUA CHAVE AQUI NOVAMENTE
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-2.5-flash-lite')

In [ ]:
def gerar_dados_clientes():
    clientes = []

    # 20 Clientes Churn (Já cancelaram)
    for i in range(1, 6):
        clientes.append({
            "id": i,
            "nome": f"Cliente {i}",
            "status_atual": "Churn",
            "dias_sem_transacao": random.randint(30, 90),
            "reclamacoes_ultimo_mes": random.randint(1, 5),
            "saldo_medio": 0
        })

    # 25 Clientes Risco (Probabilidade de Churn)
    for i in range(6, 15):
        clientes.append({
            "id": i,
            "nome": f"Cliente {i}",
            "status_atual": "Risco",
            "dias_sem_transacao": random.randint(15, 29),
            "reclamacoes_ultimo_mes": random.randint(1, 3),
            "saldo_medio": random.uniform(100, 1000)
        })

    # 55 Clientes Normais
    for i in range(15, 21):
        clientes.append({
            "id": i,
            "nome": f"Cliente {i}",
            "status_atual": "Normal",
            "dias_sem_transacao": random.randint(0, 5),
            "reclamacoes_ultimo_mes": 0,
            "saldo_medio": random.uniform(2000, 10000)
        })

    return clientes

# Salva o arquivo inicial (simulando a Extração futura)
dados_brutos = gerar_dados_clientes()
with open('clientes_brutos.json', 'w', encoding='utf-8') as f:
    json.dump(dados_brutos, f, ensure_ascii=False, indent=2)

print("Massa de dados gerada com sucesso!")

In [ ]:
# --- 1. EXTRACT (Extração) ---
print("Iniciando Extração...")
with open('clientes_brutos.json', 'r', encoding='utf-8') as f:
    dados_entrada = json.load(f)

dados_processados = []

# --- 2. TRANSFORM (Transformação com IA) ---
for i, cliente in enumerate(dados_entrada):
    plano_acao = "Cliente inativo/Churn." # Valor padrão

    # Processa apenas se NÃO for Churn
    if cliente['status_atual'] in ['Risco', 'Normal']:

        # Define o prompt
        if cliente['status_atual'] == 'Risco':
            prompt = f"Atue como um gerente de banco sênior. O cliente {cliente['nome']} está em RISCO de churn. Dados: {cliente['dias_sem_transacao']} dias sem movimentar, {cliente['reclamacoes_ultimo_mes']} reclamações recentes. Crie uma mensagem curta (máximo 20 palavras) e direta de ação para retenção."
        else:
            prompt = f"Atue como um gerente de banco. O cliente {cliente['nome']} é NORMAL e fiel. Saldo médio: R$ {cliente['saldo_medio']:.2f}. Sugira um produto de investimento em uma frase curta (máximo 20 palavras)."

        # --- Lógica de Retentativa (Retry Loop) ---
        sucesso = False
        tentativas = 0

        while not sucesso and tentativas < 3: # Tenta até 3 vezes o mesmo cliente
            try:
                # Faz a chamada à API
                response = model.generate_content(prompt)

                if response.text:
                    plano_acao = response.text.strip().replace('"', "'").replace('\n', ' ')
                else:
                    plano_acao = "Sem sugestão gerada."

                sucesso = True # Sai do loop while

                # Pausa de SUCESSO (15s para respeitar o limite de ~4 req/min do modelo novo)
                time.sleep(15)

            except Exception as e:
                erro_str = str(e)
                if "429" in erro_str:
                    print(f"⚠️ Cota excedida no cliente {cliente['id']}. Esperando 60s para esfriar...")
                    time.sleep(65) # Espera 1 minuto e 5 segundos
                    tentativas += 1
                else:
                    print(f"❌ Erro irrecuperável no ID {cliente['id']}: {erro_str}")
                    plano_acao = "Erro técnico na análise."
                    sucesso = True # Força saída para não travar
                    time.sleep(5)

    # Adiciona ao novo dataset
    novo_cliente = cliente.copy()
    novo_cliente['analise_ia'] = plano_acao
    dados_processados.append(novo_cliente)

    # Mostra progresso
    print(f"Cliente {cliente['id']} processado. Status: {cliente['status_atual']}")

# --- 3. LOAD (Carga) ---
print("Iniciando Carga...")
with open('clientes_analisados.json', 'w', encoding='utf-8') as f:
    json.dump(dados_processados, f, ensure_ascii=False, indent=2)

print("\nETL Finalizado com Sucesso! Arquivo 'clientes_analisados.json' gerado.")